# Mental health counseling ChatGPT Clone from Scratch

## Task 1: Import the Libraries

In [2]:
# Import the libraries here
import os
from datasets import load_dataset
import json
from enum import Enum
import random
from openai import OpenAI

## Task 2: Create Classes for the Dataset

In [3]:
#  Define the RoleType Enum here
class RoleType(Enum):
    USER = "user"
    SYSTEM = "system"
    ASSISTANT = "assistant"

In [4]:
# Define the Role class here
class Role(object):
    def __init__(self, role_type: RoleType, content):
        self.role = role_type.value
        self.content = content
        self.value = {'role': self.role, 'content': self.content}


In [5]:
#  Define the messsage class here
class Message(object):
    def __init__(self, user_content, system_content, assistant_content):
        self.user_role = Role(role_type = RoleType.USER, content = user_content)
        self.system_role = Role(role_type = RoleType.SYSTEM, content = system_content)
        self.assistant_role = Role(role_type = RoleType.ASSISTANT, content = assistant_content)
        self.message = {'message': [self.system_role.value, self.user_role.value, self.assistant_role.value]}


## Task 3: Load, Explore, and Store the Data

In [ ]:
# Load the dataset 
dataset = load_dataset(
    "Amod/mental_health_counseling_conversations",
    data_files = "combined_dataset.json",
    split = "train"
)

In [7]:
# Create a sample Message object
context = dataset[152]['Context']
response = dataset[152]['Response']
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."
message_obj = Message(context, system_content, response)

In [17]:
#  Create the train_dataset variable
sampled_dataset = random.choices(dataset, k=100)
train_dataset = []

for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'], system_content=system_content, assistant_content=row['Response'])
    train_dataset.append(message_obj.message)

In [19]:
# Save data in JSONl format 
def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for row in data:
            line = json.dumps(row)
            file.write(line + '\n')

training_data_path = '/usercode/data/train.jsonl'
save_to_jsonl(train_dataset[:-5], training_data_path)

validation_data_path = '/usercode/data/validation.jsonl'
save_to_jsonl(train_dataset[-5:], validation_data_path)

## Task 4: Fine-Tune the Model

In [20]:
# Load the training and validation files
training_data = open(training_data_path, "rb")
validation_data = open(validation_data_path, "rb")

In [ ]:
#  Add OpenAI api_key
api_key = ''
client = OpenAI(api_key=api_key)

In [ ]:
# Upload the training and validation files
training_response = client.files.create(file=training_data, purpose="fine-tune")
training_file_id = training_response.id

validation_response = client.files.create(file=validation_data, purpose="fine-tune")
validation_file_id = validation_response.id

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

In [ ]:
# Create a fine-tuning job
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix="my-test-model",
    validation_file=validation_file_id
)

job_id = response.id
print(response)

In [ ]:
# Retrieve the job status
job_id = response.id

job_status = client.fine_tuning.jobs.retrieve(job_id)
print(job_status)

## Task 5: Test the Fine-Tuned Model

In [31]:
# Create and store message dictionaries
system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user's emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It's important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."""

messages = []
messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. How can I fight this?"
messages.append({"role": "user", "content": user_message})

In [32]:
# Test the fine-tuned chat completion model
completion = client.chat.completions.create(
    model= response.model,
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="It's understandable to feel down during the winter season, as many people experience seasonal affective disorder (SAD) due to reduced sunlight and colder weather. Here are some strategies that may help combat these winter blues:\n\n1. **Light Therapy**: Consider using a light therapy box, which mimics natural sunlight and can help improve mood.\n   \n2. **Stay Active**: Engage in regular physical activity, even if it's just a short walk outside. Exercise can boost your mood and energy levels.\n   \n3. **Maintain a Routine**: Try to stick to a consistent daily routine to provide structure and stability.\n   \n4. **Socialize**: Stay connected with friends and family; social support can be very beneficial during challenging times.\n   \n5. **Eat Well**: Focus on a balanced diet rich in fruits, vegetables, and whole grains to support your physical and mental health.\n   \n6. **Mindfulness or Meditation**: Practicing mindfulness or meditation can help reduce s

In [33]:
# Get and compare the output of the gpt-3.5-turbo chat completion model
completion = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="I'm sorry to hear that you struggle with feeling down during the winter months. Many people experience a shift in their mood during the colder and darker seasons. One way to combat these feelings is by practicing self-care and engaging in activities that boost your mood. Here are some suggestions that might help:\n\n1. **Light Therapy**: Consider using a light therapy box to mimic natural sunlight and help regulate your mood.\n   \n2. **Stay Active**: Regular exercise can help improve your mood by releasing endorphins. Even a short walk outside can make a difference.\n\n3. **Maintain a Routine**: Stick to a regular schedule, including healthy eating habits and proper sleep hygiene.\n\n4. **Socialize**: Stay connected with friends and loved ones, even if it means virtual interactions during the winter.\n\n5. **Practice Mindfulness**: Engage in relaxation techniques, such as deep breathing, meditation, or yoga, to help manage stress and improve your mood.\n